# Lesson 3: Reflection and Blogpost Writing

## Setup

In [1]:
llm_config = {"model": "gpt-3.5-turbo"}

## The task!

In [2]:
task = '''
        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       '''


## Create a writer agent

In [3]:
import autogen

writer = autogen.AssistantAgent(
    name="Writer",
    system_message="You are a writer. You write engaging and concise " 
        "blogpost (with title) on given topics. You must polish your "
        "writing based on the feedback you receive and give a refined "
        "version. Only return your final work without additional comments.",
    llm_config=llm_config,
)

In [4]:
reply = writer.generate_reply(messages=[{"content": task, "role": "user"}])

In [5]:
print(reply)

Title: "Exploring the World of DeepLearning.AI"

Dive into the realm of DeepLearning.AI, a cutting-edge platform revolutionizing artificial intelligence. From beginner to advanced courses, DeepLearning.AI offers invaluable insights into neural networks, computer vision, and more. Sharpen your skills with hands-on projects and gain expertise from top industry experts like Andrew Ng. Whether you're a tech enthusiast or a seasoned professional, this platform provides a comprehensive learning experience tailored to your needs. Join the AI revolution today and unlock a world of possibilities with DeepLearning.AI.


## Adding reflection 

Create a critic agent to reflect on the work of the writer agent.

In [6]:
critic = autogen.AssistantAgent(
    name="Critic",
    is_termination_msg=lambda x: x.get("content", "").find("TERMINATE") >= 0,
    llm_config=llm_config,
    system_message="You are a critic. You review the work of "
                "the writer and provide constructive "
                "feedback to help improve the quality of the content.",
)

In [7]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of DeepLearning.AI"

Dive into the realm of DeepLearning.AI, a cutting-edge platform revolutionizing artificial intelligence. From beginner to advanced courses, DeepLearning.AI offers invaluable insights into neural networks, computer vision, and more. Sharpen your skills with hands-on projects and gain expertise from top industry experts like Andrew Ng. Whether you're a tech enthusiast or a seasoned professional, this platform provides a comprehensive learning experience tailored to your needs. Join the AI revolution today and unlock a world of possibilities with DeepLearning.AI.

--------------------------------------------------------------------------------
Critic (to Writer):

Your blog post title,

## Nested chat

In [8]:
SEO_reviewer = autogen.AssistantAgent(
    name="SEO Reviewer",
    llm_config=llm_config,
    system_message="You are an SEO reviewer, known for "
        "your ability to optimize content for search engines, "
        "ensuring that it ranks well and attracts organic traffic. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)


In [9]:
legal_reviewer = autogen.AssistantAgent(
    name="Legal Reviewer",
    llm_config=llm_config,
    system_message="You are a legal reviewer, known for "
        "your ability to ensure that content is legally compliant "
        "and free from any potential legal issues. "
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role.",
)

In [10]:
ethics_reviewer = autogen.AssistantAgent(
    name="Ethics Reviewer",
    llm_config=llm_config,
    system_message="You are an ethics reviewer, known for "
        "your ability to ensure that content is ethically sound "
        "and free from any potential ethical issues. " 
        "Make sure your suggestion is concise (within 3 bullet points), "
        "concrete and to the point. "
        "Begin the review by stating your role. ",
)

In [11]:
meta_reviewer = autogen.AssistantAgent(
    name="Meta Reviewer",
    llm_config=llm_config,
    system_message="You are a meta reviewer, you aggragate and review "
    "the work of other reviewers and give a final suggestion on the content.",
)

## Orchestrate the nested chats to solve the task

In [12]:
def reflection_message(recipient, messages, sender, config):
    return f'''Review the following content. 
            \n\n {recipient.chat_messages_for_summary(sender)[-1]['content']}'''

review_chats = [
    {
     "recipient": SEO_reviewer, 
     "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}. Here Reviewer should be your role",},
     "max_turns": 1},
    {
    "recipient": legal_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'Reviewer': '', 'Review': ''}.",},
     "max_turns": 1},
    {"recipient": ethics_reviewer, "message": reflection_message, 
     "summary_method": "reflection_with_llm",
     "summary_args": {"summary_prompt" : 
        "Return review into as JSON object only:"
        "{'reviewer': '', 'review': ''}",},
     "max_turns": 1},
     {"recipient": meta_reviewer, 
      "message": "Aggregrate feedback from all reviewers and give final suggestions on the writing.", 
     "max_turns": 1},
]


In [13]:
critic.register_nested_chats(
    review_chats,
    trigger=writer,
)

**Note**: You might get a slightly different response than what's shown in the video. Feel free to try different task.

In [14]:
res = critic.initiate_chat(
    recipient=writer,
    message=task,
    max_turns=2,
    summary_method="last_msg"
)

Critic (to Writer):


        Write a concise but engaging blogpost about
       DeepLearning.AI. Make sure the blogpost is
       within 100 words.
       

--------------------------------------------------------------------------------
Writer (to Critic):

Title: "Exploring the World of DeepLearning.AI"

Dive into the realm of DeepLearning.AI, a cutting-edge platform revolutionizing artificial intelligence. From beginner to advanced courses, DeepLearning.AI offers invaluable insights into neural networks, computer vision, and more. Sharpen your skills with hands-on projects and gain expertise from top industry experts like Andrew Ng. Whether you're a tech enthusiast or a seasoned professional, this platform provides a comprehensive learning experience tailored to your needs. Join the AI revolution today and unlock a world of possibilities with DeepLearning.AI.

--------------------------------------------------------------------------------

*****************************************


--------------------------------------------------------------------------------
Meta Reviewer (to Critic):

Aggregate Feedback:
- All reviewers emphasize the importance of including specific keywords related to DeepLearning.AI to enhance search visibility.
- They all recommend incorporating internal links to relevant pages within the content to improve site structure and enhance user experience.
- Craft a meta title and description that accurately reflect the content and include primary keywords to boost click-through rates in search results.

Final Suggestions:
Based on the feedback provided by the reviewers, it is crucial to focus on implementing the suggested SEO strategies. Incorporating specific keywords related to DeepLearning.AI, internal links, and optimizing meta tags will not only improve the visibility of the content but also enhance the overall user experience. By following these recommendations, the content can potentially attract more traffic and engagement.

----------

## Get the summary

In [15]:
print(res.summary)

Title: "Master DeepLearning.AI: Your Gateway to AI Expertise"

Meta Title: "Unlock AI Insights with DeepLearning.AI | Courses by Andrew Ng"
Meta Description: "Explore the top-rated DeepLearning.AI platform for neural networks and computer vision. Gain expertise and hands-on experience with industry-leading courses by Andrew Ng. Start your AI journey today."

Blogpost:
Immerse yourself in the world of DeepLearning.AI, the premier platform reshaping artificial intelligence. From neural networks to computer vision, DeepLearning.AI provides a comprehensive learning experience tailored to tech enthusiasts and professionals alike. Delve into hands-on projects and learn from top industry experts, including the renowned Andrew Ng. Elevate your skills and join the AI revolution today with DeepLearning.AI — where possibilities are limitless.
